In [1]:
import json
import requests
import urllib3
import math

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

AUTH_LOGIN_URL = 'https://pse-console-auth.intel.com/api/v1/aad/login'
SYSTEM_LIST_URL = 'https://pse-console-data.intel.com/api/v1/inventory/systems'
SHORT_LIST_URL = 'https://pse-console-query-bot-relay.intel.com/api/v1/mongodb/downloads_short'
TIMEOUT = 3600

def auth_login(email, password):
    headers = {
        'accept': 'application/json'
    }

    data = {
        'email': f'{email}',
        'password': f'{password}',
    }

    try:
        response = requests.post(
            AUTH_LOGIN_URL,
            headers=headers,
            verify=False,  # nosec B501
            json=data,
            timeout=TIMEOUT,
        )

        response.raise_for_status()
        system_info_response_json = response.json()

        print('Successfully authorized')
        return True, system_info_response_json['access_token']
    except requests.RequestException as e:
        print('Could not authorize', e)
        return False, ''
    except Exception as e:
        print('Unexpected error during auth', e)
        return False, ''

def get_all_system_ids(token):
    headers = {
        'accept': 'application/json',
        'Authorization': f'Bearer {token}',
    }

    data = {
        'attributes_filter': {},
        'system_filters': [],
        'system_ids': [],
    }

    try:
        response = requests.post(
            'https://pse-console-data.intel.com/api/v1/systems/systems-filter?system_id_only=true',
            headers=headers,
            verify=False,  # nosec B501
            json=data,
            timeout=TIMEOUT,
        )

        response.raise_for_status()
        system_list = response.json()

        return True, system_list
    except requests.RequestException as e:
        print('Could not fetch system ids %s', e)
        return False, []
    except Exception as e:
        print('Unexpected error during gathering system ids %s', e)
        return False, []

def get_short_list(token, system_ids):
    headers = {
        'accept': 'application/json',
        'Authorization': f'Bearer {token}',
    }
    #print(system_ids)
    data = {
        "system_ids": system_ids,
        "field_names" : [
                "System Name",
                "System Type",
                "CPU",
                "Memory",
                "BIOS",
                "Board Product",
                "HW Config",
                "Board Serial",
                "PBA",
                "SuperGroup",
                "Group",
                "Team",
                "Division",
                "Subdivision",
                 "Discipline",
                 "Val Team",
                "Function",
                "Pool",
                "Product",
                 "Project",
                 "Processor",
                 "Domain",
                "Activity",
                "Owner",
                "Active State",
                "System State",
                "NGA Project",
                "NGA Activity",
                "NGA Config",
                "Last Scan",
                "BAT Result",
                "HMA Version",
                "Location",
                "HMA Status"
                ]
    }

    try:
        response = requests.post(
            SHORT_LIST_URL,
            headers=headers,
            verify=False,  # nosec B501
            json=data,
            timeout=TIMEOUT,
            stream=True
        )

        response.raise_for_status()
        print("Hello world")
        print(SHORT_LIST_URL)
        print (response.content)
        
        
        
        with open('PSEDATA.csv', 'wb') as f:
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)

        print('Wrote to shortlist.csv')

        return True
    except requests.RequestException as e:
        print('Could not get short list', e)
        return False
    except Exception as e:
        print('Unexpected error during short list', e)
        return False

def main(email, password):
    success, token = auth_login(email, password)

    if not success:
        print('Cannot continue with invalid auth')
        exit(1)

    print('Acquired access token')

    success, system_ids = get_all_system_ids(token)

    print('system id count: ', len(system_ids))

    if success:
        get_short_list(token, system_ids)

if __name__ == '__main__':
    import sys
    email = sys.argv[1]
    password = sys.argv[2]

    main("oscar.hernandez.martinez@intel.com", "Economos48.")



Successfully authorized
Acquired access token
system id count:  11672
Hello world
https://pse-console-query-bot-relay.intel.com/api/v1/mongodb/downloads_short


IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [7]:
import requests
url = "https://pse-console-auth.intel.com/api/v1/aad/login"
payload = {
    "email": "oscar.hernandez.martinez@intel.com",
    "password": "Economos48."
}
response = requests.post(url, json=payload, verify=False, timeout=10)
response.raise_for_status()
if response.status_code == 200:
    token = response.json().get("access_token")
else:
    print ("Failed to retrieve token")
print(token)

eyJ0eXAiOiJKV1QiLCJhbGciOiJSUzI1NiIsIng1dCI6IkpZaEFjVFBNWl9MWDZEQmxPV1E3SG4wTmVYRSIsImtpZCI6IkpZaEFjVFBNWl9MWDZEQmxPV1E3SG4wTmVYRSJ9.eyJhdWQiOiJodHRwczovL2V2ZmQuaW50ZWwuY29tIiwiaXNzIjoiaHR0cHM6Ly9zdHMud2luZG93cy5uZXQvNDZjOThkODgtZTM0NC00ZWQ0LTg0OTYtNGVkNzcxMmUyNTVkLyIsImlhdCI6MTc1NTExNjMzMiwibmJmIjoxNzU1MTE2MzMyLCJleHAiOjE3NTUxMjEyMjEsImFjciI6IjEiLCJhaW8iOiJBWFFBaS84WkFBQUFHTVk1NFZxNktLejNYdG1POGdKbDc4RC80WkJjajFaeTR4cjVTRFdRRXI1RnF2b3R4YjlqcHdVRVArN2Zmc0FJYkhrTnk4UVJTUGh0SWhlUk51eHVyWUM0RmJtMHBEQmZYRU1jNWJWRHJzNUxOTk1HVUJJejVUVEI2bFU0L0Y3dk9EU2hnOTZPS0JXTW8wNVdvRVQ3SXc9PSIsImFtciI6WyJwd2QiXSwiYXBwaWQiOiI5NzVmN2RhNy0xYTk0LTQ0MDgtODYwNS1iMzYwMjNmZDkwMWUiLCJhcHBpZGFjciI6IjEiLCJmYW1pbHlfbmFtZSI6Ikhlcm5hbmRleiBNYXJ0aW5leiIsImdpdmVuX25hbWUiOiJPc2NhciIsImdyb3VwcyI6WyI5YjI1OTRiZS00MzE2LTRjMTUtYWM0Zi0wMTg2YWU5ZTI5OTgiXSwiaXBhZGRyIjoiMTk4LjE3NS42OC41MiIsIm5hbWUiOiJIZXJuYW5kZXogTWFydGluZXosIE9zY2FyIiwib2lkIjoiYzY4YTJlMGUtZGY2Mi00MGY5LWFhOWUtMDA5YThiYjNiODNhIiwib25wcmVtX3NpZCI6IlMtMS01LTIxLTcyNTM